In [3]:
#! pip install beautifulsoup4, requests
# !pip install selenium
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import numpy as np
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support import ui
import os
import sys
os.path.dirname(sys.executable)
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time

In [95]:
url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Milwaukee%2C%20WI&start=230"
html = requests.get(url)
html.content[0:500]

b'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel='

In [96]:
# Making the soup
from bs4 import BeautifulSoup
soup = BeautifulSoup(html.content, 'lxml')

In [97]:
#soup.head()

In [98]:
# Identifying the restaurant links
links = soup.select(".css-166la90")
print()

# Getting the restaurant names and the restaurant links in solation
name_restaurant = []
link_restaurant = []

for link in links:
    name_restaurant.append(link.string)
    link_restaurant.append(link['href'])
    
print(name_restaurant[0:10])


['Bartolotta’s Lake Park Bistro', 'Riley’s Sandwich Co', 'MidTown Grill', 'Milwaukee Sail Loft', 'Mader’s', 'Vagabond', 'Rare Steakhouse', 'Urban Beets Cafe & Juicery', 'Shake Shack', 'Taco Mikes']


In [150]:
# Getting everything also for multiple pages (here, for page 2, 3, 4, 5, and 6)
url_yelp = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Milwaukee%2C%20WI&start="


# we initialize the links
full_links = []
full_restaurants = []

for i in range(0,240,10):
    
    html = requests.get(url_yelp+str(i))
    
    soup = BeautifulSoup(html.content, 'lxml')
    link = soup.select(".css-166la90")
    
    
    name_restaurant = []
    link_restaurant = []

   
    for l in link:
        name_restaurant.append(l.string)
        link_restaurant.append(l.get('href'))
     
        #Here we only ectract the first 10 entries
    full_restaurants.extend(name_restaurant[0:10])
    full_links.extend(link_restaurant[0:10])
    
print(full_restaurants)

['Swingin’ Door Exchange', 'Story Hill BKC', 'Mad Rooster Cafe', 'Maxie’s', 'View MKE', 'Tusk', 'Triciclo Peru', 'St Paul Fish Company', 'Lazo’s Taco Shack', 'Five O’Clock Steakhouse', 'Blue Bat Kitchen and Tequilaria', 'Odd Duck', 'Public Table', 'Third Coast Provisions', 'Fool’s Errand', 'The Capital Grille', 'SweetDiner', 'Blue’s Egg - Milwaukee', 'Amilinda', 'Toast', 'Harbor House', 'La Merenda', 'Sobelman’s Pub & Grill', 'Uncle Wolfie’s Breakfast Tavern', 'Smoke Shack', 'Tangled Noodles', 'Momo Mee', 'Camino', 'The Tandem', 'MOVIDA - Hotel Madrid', 'Char’d', 'DanDan', 'The Noble', 'Le Reve Patisserie & Café', 'Tavo’s Signature Cuisine', 'Ca’Lucchenzo', 'Lowcountry Milwaukee', 'Morel', 'The Diplomat', 'The Explorium Brewpub Third Ward', 'Oscar’s Pub & Grill', 'Bavette', 'San Giorgio Pizzeria Napoletana', 'Buckley’s Restaurant & Bar', 'Stella Van Buren', 'Oak Barrel Public House', 'The Fitz', 'The Chef’s Table', 'Tied House Milwaukee', 'Café Benelux', 'Birch + Butcher', 'The Origina

In [151]:
len(full_restaurants)

240

In [152]:
#then we zip it together and store it in a dataframe
entry_list = list(zip(full_restaurants, full_links))
            
df_mainlinks = pd.DataFrame(entry_list,columns=['restaurant', 'link'])

In [153]:
df_mainlinks

,restaurant,link
0,Swingin’ Door Exchange,/biz/swingin-door-exchange-milwaukee?osq=Resta...
1,Story Hill BKC,/biz/story-hill-bkc-milwaukee?osq=Restaurants
2,Mad Rooster Cafe,/biz/mad-rooster-cafe-west-milwaukee?osq=Resta...
3,Maxie’s,/biz/maxies-milwaukee?osq=Restaurants
4,View MKE,/biz/view-mke-milwaukee?osq=Restaurants
...,...,...
235,Valley Inn,/biz/valley-inn-milwaukee?osq=Restaurants
236,Rice N Roll Bistro,/biz/rice-n-roll-bistro-milwaukee?osq=Restaurants
237,Crave Cafe,/biz/crave-cafe-shorewood?osq=Restaurants
238,Deli 1614,/biz/deli-1614-milwaukee?osq=Restaurants


In [154]:
df_mainlinks.to_csv('mainlinks.csv')

In [2]:
df = pd.read_csv('links.csv')
df_mainlinks = pd.read_csv('mainlinks.csv')

In [3]:
''' Now we need to comapre the difference'''

old_rest = df.Restaurant.to_list()

#new_rest = df_mainlinks.restaurant.to_list()

df_new = df_mainlinks[~df_mainlinks['restaurant'].isin(old_rest)]

In [4]:
df_new.drop_duplicates(subset= ['restaurant'], inplace=True)

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
#let's us regex to clean the df
import re

In [6]:
#we convert everyting to strings, and then exclude those rows where the column Links contains a https
df_new['restaurant'] = df_new.restaurant.astype('str') 
df_new['link'] = df_new.link.astype('str') 

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
'''then we complete the links colum for the full URL'''

df_new['Complete_links']= df_new['link'].apply(lambda x:"https://www.yelp.com"+str(x))

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df_new.pop('link')

79          /biz/black-sheep-milwaukee-6?osq=Restaurants
98       /biz/taste-amirs-roti-milwaukee?osq=Restaurants
101           /biz/wild-roots-west-allis?osq=Restaurants
110          /biz/the-social-milwaukee-2?osq=Restaurants
114            /biz/easy-tyger-milwaukee?osq=Restaurants
120      /biz/the-wicked-hop-milwaukee-2?osq=Restaurants
121            /biz/troop-cafe-milwaukee?osq=Restaurants
127          /biz/two-brothers-milwaukee?osq=Restaurants
130       /biz/caradaro-club-milwaukee-2?osq=Restaurants
131    /biz/kin-by-rice-n-roll-wauwatosa-2?osq=Restau...
133    /biz/restaurant-to-be-named-later-milwaukee?os...
136          /biz/carsons-ribs-milwaukee?osq=Restaurants
140       /biz/bottle-house-42-milwaukee?osq=Restaurants
141             /biz/mistral-milwaukee-2?osq=Restaurants
145        /biz/siam-thai-west-milwaukee?osq=Restaurants
146          /biz/firefly-tosa-wauwatosa?osq=Restaurants
156    /biz/north-avenue-grill-wauwatosa?osq=Restaurants
157    /biz/milwaukee-steakhous

In [9]:
len(df_new) 

45

In [70]:
# we see that there are two duplicate restaurants, which we can drop subsequnetly
df.groupby(['Restaurant']).size().sort_values(ascending=False)

Restaurant
Egg & Flour Pasta Bar     2
Camino                    2
2Mesa Mexican Eatery      1
Shake Shack               1
Riley’s Sandwich Co       1
                         ..
Goodkind                  1
Guadalajara Restaurant    1
Hamburger Mary’s          1
Harbor House              1
Zarletti                  1
Length: 208, dtype: int64

In [76]:
df.drop_duplicates(keep='first', inplace=True)

In [78]:
df.to_csv('links.csv',index=False,header=True,encoding='utf8')

### Next, we scrape all the details about the restaurants with a web driver and the selenium package

In [85]:
df = pd.read_csv('links.csv')

In [86]:
df.tail()

,Restaurant,Complete_links
205,La Fondita De Milwaukee,https://www.yelp.com/biz/la-fondita-de-milwauk...
206,Taqueria Buenavista,https://www.yelp.com/biz/taqueria-buenavista-m...
207,Michael’s Family Restaurant,https://www.yelp.com/biz/michaels-family-resta...
208,Triskele’s,https://www.yelp.com/biz/triskeles-milwaukee?o...
209,El Beso Milwaukee - Mexican Restaurante & Cantina,https://www.yelp.com/biz/el-beso-milwaukee-mex...


In [140]:
'''options to avoide that actual windows are opened every time. create headless driver options'''
options = webdriver.FirefoxOptions()

#these options are set so that a browser doesn't actually open everytime we scrape
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

In [5]:
Links = df.Complete_links.to_list() #in the formula below Links is equal to Li
Restaurants = df.Restaurant.to_list() #in the formula below Restaurants is equal to Rest

In [141]:
''' Here we can make a little test run'''

time.sleep(3)
driver = webdriver.Firefox() #options=options 
driver.get('https://www.yelp.com/biz/highlands-cafe-wauwatosa?osq=Restaurants')

driver.maximize_window()
time.sleep(3)

#we first need to get rid of the privacy concern button so we click it
driver.find_element_by_css_selector('.onetrust-lg').click()

#then we wait until that click is processed, before we click the button where all the amenties are extended
time.sleep(3)
try:
    driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(6) > div:nth-child(2) > button:nth-child(2)').click()
except:
     print('no hidden amenities')

try:
    driver.find_element_by_css_selector(' div.padding-t3__373c0__1gw9E:nth-child(3) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)').click()
    time.sleep(3)
    driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(5) > div:nth-child(2) > button:nth-child(2)').click()
    time.sleep(2)
    print('child 5')
    
except:
    print('no ammenities')
    

source = driver.page_source
soup1 = BeautifulSoup(source, 'html.parser')

no ammenities


In [24]:
driver.quit()

In [25]:
Li = df_new.Complete_links.to_list()
Rest = df_new.restaurant.to_list()

In [26]:
Li =Li[10:]
Rest = Rest[10:]

In [29]:
button.css-174jypu:nth-child(2)

'Restaurant To Be Named Later'

In [30]:
'''Selenium search automation can either fetch the search item from the list or text can be entered'''

df_rest = pd.DataFrame([],columns=['restaurant', 'link', 'claimed', 'number_of_reviews', 'number_of_photos', 'price', 'rating', 'categories', 'street', 'address', 'website', 'update', 'amenities'])


#This is so we can enumerate over the restaurant names
enum_extract = enumerate(Rest)

def getname():
    key = next(enum_extract)
    return str(key)

enum_rest = enumerate(Li)

def getlink():
    key = next(enum_rest)
    return str(key)
    


for i in Li:
    restaurant_link = []
    restaurant_update = []
    restaurant_description = []
    restaurant_numberofreviews = []        
    restaurant_photos = []
    restaurant_rating = []
    restaurant_street = []
    restaurant_adress = []

    restaurant_link = []
    restaurant_price = []
    restaurant_categories = []
    restaurant_name = [] 

    restaurant_claimed = [] 
    restaurant_amenities = []

    #so here we append the restaurant name
    restname = getname()
    restaurant_name.append(restname) 
    
    restlink = getlink()
    restaurant_link.append(restlink)
    
    try:
            #Here we initialize a headless driver
        time.sleep(3)
        driver = webdriver.Firefox(options=options) #options=options 
        driver.get(i)

        driver.maximize_window()
        time.sleep(3)

        #we first need to get rid of the privacy concern button so we click it
        driver.find_element_by_css_selector('.onetrust-lg').click()

        #then we wait until that click is processed, before we click the button where all the amenties are extended
        time.sleep(3)
        
        try:
            driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(6) > div:nth-child(2) > button:nth-child(2)').click()
            time.sleep(3)
        except:
            print('no hidden amenities')
        
        try:
            driver.find_element_by_css_selector(' div.padding-t3__373c0__1gw9E:nth-child(3) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)').click()
            time.sleep(3)
            driver.find_element_by_css_selector(' section.margin-t4__373c0__1TRkQ:nth-child(5) > div:nth-child(2) > button:nth-child(2)').click()
            time.sleep(2)
            print('child 5')
        
        except:
            print('no ammenities')


        source = driver.page_source
        soup1 = BeautifulSoup(source, 'html.parser')


        #now we loop over the elements in the class
        subsoup = soup1.find_all(class_ = 'arrange-unit__373c0__1piwO border-color--default__373c0__2oFDT')
        amenities = []
        for k in range(len(subsoup)):
            subamenities=subsoup[k].get_text()
            amenities.append(subamenities)

        restaurant_amenities.append(amenities) 


        #number of reviews
        numberofreviews = soup1.select(".css-bq71j2") 
        restaurant_numberofreviews.append(numberofreviews[0].string)

        #number of photos
        photos = soup1.select(".css-ardur")
        restaurant_photos.append(photos[4].string) 

        #first and second address line
        location = soup1.select(".raw__373c0__3rcx7")
        restaurant_street.append(location[0].string) 
        restaurant_adress.append(location[1].string)

            #rating
        for item in soup1.select('[class*=container]'):
            if item.find('h4'):
                name = item.find('h4').get_text()
                ratings=soup1.select('[aria-label*=rating]')[0]
                rating=str(ratings)
                rating = re.search('aria-label="(.+?)star rating"', rating)
                if rating:
                    rating = rating.group(1)

        restaurant_rating.append(rating)

        #price
        price = soup1.select(".css-1xxismk")
        price=str(price)
        price = re.search('"css-1xxismk">(.+?)<', price)
        if price:
            price = price.group(1)
            restaurant_price.append(price) 

        #categories
        #this list needs to be inside the loop
        categories = []

        for item in soup1.select('[class*=container]'):
            if item.find('h1'):
                items=item.select('.css-166la90')
                for i in items:
                    categories.append(i.string)

                restaurant_categories.append(categories)

        if 'Unclaimed' not in categories:

            #updates
            update=soup1.select('.css-gdi06s')
            restaurant_update.append(update[0].string)             


            #homepage link
            link = soup1.select(".css-ac8spe")
            restaurant_link.append(link[6].string)


            restaurant_claimed.append('Claimed')

        else: 
            restaurant_update.append('None')
            restaurant_link.append('None')
            restaurant_claimed.append('Unclaimed')

            driver.quit()


    except: 
        restaurant_claimed.append('None')
        restaurant_update.append('None')
        restaurant_numberofreviews.append('None')
        restaurant_photos.append('None')
        restaurant_rating.append('None')
        restaurant_street.append('None')
        restaurant_adress.append('None')
        restaurant_link.append('None')
        restaurant_price.append('None')
        restaurant_categories.append('None')
        restaurant_amenities.append('None')
            
    finally:
        if len(restaurant_claimed) == 0:
            restaurant_claimed.append('None')
                
        if len(restaurant_update) == 0:
            restaurant_update.append('None')
                
        if len(restaurant_numberofreviews) == 0:
            restaurant_numberofreviews.append('None')
                
        if len(restaurant_photos) == 0:
            restaurant_photos.append('None')
                
        if len(restaurant_rating) == 0:
            restaurant_rating.append('None')
                
        if len(restaurant_street) == 0:
            restaurant_street.append('None')
                
        if len(restaurant_adress) == 0:
            restaurant_adress.append('None')
                
        if len(restaurant_link) == 0:
            restaurant_link.append('None')
                
        if len(restaurant_price) == 0:
            restaurant_price.append('None')
                
        if len(restaurant_categories) == 0:
            restaurant_categories.append('None')
                
        if len(restaurant_amenities) == 0:
            restaurant_amenities.append('None')
                
        entry_list = list(zip(restaurant_name, restaurant_link, restaurant_claimed, restaurant_numberofreviews, restaurant_photos, restaurant_price, restaurant_rating, restaurant_categories, restaurant_street, restaurant_adress, restaurant_link,restaurant_update, restaurant_amenities))

            
        df_entry = pd.DataFrame(entry_list,columns=['restaurant', 'link', 'claimed', 'number_of_reviews', 'number_of_photos', 'price', 'rating', 'categories', 'street', 'address', 'website', 'update', 'amenities'])
            
        frames = [df_rest, df_entry]
        df_rest = pd.concat(frames)
        time.sleep(5)

no hidden amenities
no ammenities
no ammenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no ammenities
no ammenities
child 5
no ammenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
child 5
no ammenities
no ammenities
no hidden amenities
no ammenities
no ammenities
no ammenities
no ammenities
no ammenities
no hidden amenities
no ammenities
no hidden amenities
no ammenities


In [103]:
#with this code we can always check whether we got blocked by yelp, the status code should be equal to 200
res  = requests.get('https://www.yelp.com/biz/lazos-taco-shack-milwaukee?osq=Restaurants')

res.status_code

200

### We scraped the restaurants in several gos, which is why we now have to concatinate the different frames and clean them up

In [45]:
len(df_new) - (len(df9) + len(df10))

19

In [32]:
df_rest.tail()

,restaurant,link,claimed,number_of_reviews,number_of_photos,price,rating,categories,street,address,website,update,amenities
0,"(30, 'Skyway Cafe')","(30, 'https://www.yelp.com/biz/skyway-cafe-mil...",Unclaimed,13 reviews,See 23 photos,$,4.5,"[Unclaimed, Cafes]",626 East Wisconsin Ave,Ground floor,"(30, 'https://www.yelp.com/biz/skyway-cafe-mil...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(31, 'The Green Kitchen')","(31, 'https://www.yelp.com/biz/the-green-kitch...",Unclaimed,164 reviews,See 77 photos,$,4,"[Unclaimed, Vegetarian, Sandwiches]",400 N Water St,"Milwaukee, WI 53202","(31, 'https://www.yelp.com/biz/the-green-kitch...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(32, 'Thainamite - Wauwatosa')","(32, 'https://www.yelp.com/biz/thainamite-wauw...",Claimed,141 reviews,See 134 photos,$$,3.5,"[Thai, Sushi Bars, Cocktail Bars]",8725 W North Ave,"Wauwatosa, WI 53226","(32, 'https://www.yelp.com/biz/thainamite-wauw...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(33, 'Deli 1614')","(33, 'https://www.yelp.com/biz/deli-1614-milwa...",Unclaimed,17 reviews,See 12 photos,$,4.5,"[Unclaimed, Delis]",1614 W Walnut St,"Milwaukee, WI 53205","(33, 'https://www.yelp.com/biz/deli-1614-milwa...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."
0,"(34, 'Riverfront Pizzeria Bar & Grill')","(34, 'https://www.yelp.com/biz/riverfront-pizz...",None,136 reviews,See 110 photos,$$,3.5,"[Pizza, Bars]",509 E Erie St,"Milwaukee, WI 53202","(34, 'https://www.yelp.com/biz/riverfront-pizz...",None,"[, For BusinessesWrite a ReviewLog InSign Up, ..."


In [34]:
df10 = df_rest

In [35]:
df10.to_csv('usable10.csv')

In [36]:
df1 = pd.read_csv('usable.csv')
df2 = pd.read_csv('usable2.csv')
df3 = pd.read_csv('usable3.csv')
df4 = pd.read_csv('usable4.csv')
df5 = pd.read_csv('usable5.csv')
df6 = pd.read_csv('usable6.csv')
df7 = pd.read_csv('usable7.csv')
df8 = pd.read_csv('usable8.csv')
df9 = pd.read_csv('usable9.csv')
df10 = pd.read_csv('usable10.csv')

In [48]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]
dfdf = pd.concat(frames)

In [49]:
len(dfdf)

297

In [50]:
dfdf.head()
dfdf = dfdf.loc[:, ~dfdf.columns.str.contains('^Unnamed')]
dfdf['restaurant'] = dfdf['restaurant'].astype('str') 

In [53]:
import regex as re

new_restaurants = dfdf.restaurant.to_list()

'''stripping anything werid from the search term'''
def searchconcat(x):
    x = x.replace(',', '')\
    .replace("'","")\
    .replace('(','')\
    .replace(')','')\
    .replace('0', '')\
    .replace('1', '')\
    .replace('2', '')\
    .replace('3', '')\
    .replace('4', '')\
    .replace('5', '')\
    .replace('6', '')\
    .replace('7', '')\
    .replace('8', '')\
    .replace('9', '')\
    .strip()
    
    return str(x)



new_ii = []
for i in new_restaurants:
    new_i=searchconcat(i)
    new_ii.append(new_i)
    

dfdf['restaurant'] = new_ii

In [56]:
dfdf.drop_duplicates(subset=['restaurant'], keep='last', inplace = True)

In [62]:
len(dfdf)

253

### Then here is the final dataframe

In [67]:
dfdf.tail(50)

,restaurant,link,claimed,number_of_reviews,number_of_photos,price,rating,categories,street,address,website,update,amenities
6,Tiki Bar MKE Public Market,"(6, 'https://www.yelp.com/biz/tiki-bar-mke-pub...",Unclaimed,5 reviews,See 29 photos,None,5.0,"['Unclaimed', 'Bars', 'Seafood']",401 N Broadway,"Milwaukee, WI 53202","(6, 'https://www.yelp.com/biz/tiki-bar-mke-pub...",None,"['', 'For BusinessesWrite a ReviewLog InSign U..."
7,Alem Ethiopian Village,"(7, 'https://www.yelp.com/biz/alem-ethiopian-v...",Claimed,279 reviews,See 147 photos,$$,4.5,"['Ethiopian', 'Vegetarian']",307 E Wisconsin Ave,"Milwaukee, WI 53202","(7, 'https://www.yelp.com/biz/alem-ethiopian-v...",NaN,"['', 'For BusinessesWrite a ReviewLog InSign U..."
8,East Town Kitchen & Bar,"(8, 'https://www.yelp.com/biz/east-town-kitche...",Claimed,75 reviews,See 120 photos,$$,4.0,"['American (New)', 'Beer Bar', 'Cocktail Bars']",323 E Wisconsin Ave,"Milwaukee, WI 53202","(8, 'https://www.yelp.com/biz/east-town-kitche...",Milwaukee Marriott Downtown,"['', 'For BusinessesWrite a ReviewLog InSign U..."
9,Festival Foods,"(9, 'https://www.yelp.com/biz/festival-foods-m...",Claimed,4 reviews,See 43 photos,None,3.5,"['Grocery', 'Bakeries', 'Delis']",11111 W Greenfield Ave,"Milwaukee, WI 53214","(9, 'https://www.yelp.com/biz/festival-foods-m...",NaN,"['', 'For BusinessesWrite a ReviewLog InSign U..."
11,Egg & Flour Pasta Bar,"(11, 'https://www.yelp.com/biz/egg-and-flour-p...",Unclaimed,38 reviews,See 98 photos,None,4.5,"['Unclaimed', 'Italian']",2273 S Howell Ave,"Milwaukee, WI 53207","(11, 'https://www.yelp.com/biz/egg-and-flour-p...",None,"['', 'For BusinessesWrite a ReviewLog InSign U..."
0,Black Sheep,"(0, 'https://www.yelp.com/biz/black-sheep-milw...",Claimed,256 reviews,See 360 photos,$$,4,"['Wine Bars', 'American (New)', 'Breakfast & B...",216 S 2nd St,"Milwaukee, WI 53204","(0, 'https://www.yelp.com/biz/black-sheep-milw...","""Open for dine-in, carryout, curbside and deli...","['', 'For BusinessesWrite a ReviewLog InSign U..."
1,Taste Amir’s Roti,"(1, 'https://www.yelp.com/biz/taste-amirs-roti...",Unclaimed,2 reviews,See 4 photos,None,5,"['Unclaimed', 'Food Trucks', 'Indian']",3953 S 20th St,"Milwaukee, WI 53221","(1, 'https://www.yelp.com/biz/taste-amirs-roti...",None,"['', 'For BusinessesWrite a ReviewLog InSign U..."
2,Wild Roots,"(2, 'https://www.yelp.com/biz/wild-roots-west-...",Claimed,72 reviews,See 134 photos,$$,4.5,"['Breakfast & Brunch', 'American (New)', 'Beer...",6807 W Becher St,"West Allis, WI 53219","(2, 'https://www.yelp.com/biz/wild-roots-west-...",NaN,"['', 'For BusinessesWrite a ReviewLog InSign U..."
3,The Social,"(3, 'https://www.yelp.com/biz/the-social-milwa...",Unclaimed,1 review,See 6 photos,None,5,"['Unclaimed', 'American (Traditional)', 'Beer ...",611 N Broadway,"Milwaukee, WI 53202","(3, 'https://www.yelp.com/biz/the-social-milwa...",None,"['', 'For BusinessesWrite a ReviewLog InSign U..."
4,Easy Tyger,"(4, 'https://www.yelp.com/biz/easy-tyger-milwa...",Claimed,144 reviews,See 226 photos,$$,4.5,"['Ramen', 'Asian Fusion', 'Gastropubs']",1230 E Brady St,"Milwaukee, WI 53202","(4, 'https://www.yelp.com/biz/easy-tyger-milwa...",NaN,"['', 'For BusinessesWrite a ReviewLog InSign U..."


In [63]:
dfdf.to_csv('milwaukee.restaurants.yelp.csv')

In [66]:
dflist =dfdf.restaurant.to_list()
len(set(dflist))

253

### Now on to the individual reviews

In [4]:
df = pd.read_csv('mainlinks.csv')

In [5]:
'''then we complete the links colum for the full URL'''

df['Complete_links']= df['link'].apply(lambda x:"https://www.yelp.com"+str(x))

In [7]:
Links = df.Complete_links.to_list() #in the formula below Links is equal to Li
Restaurants = df.restaurant.to_list() #in the formula below Restaurants is equal to Rest

In [47]:
restrest = Restaurants[90:]
linklink = Links[90:]

In [48]:
restrest[0:5]

['Aperitivo',
 'Maki Yaki Japanese Grill',
 'Proof Pizza',
 'Tua Pasta',
 'Who’s On Third']

In [49]:
url_yelp = 'https://www.yelp.com/biz/sweetdiner-milwaukee?osq=Restaurants'
html = requests.get(url_yelp)
soup2 = BeautifulSoup(html.content, 'lxml')

html.status_code

200

In [64]:
df[df['Complete_links'] == 'https://www.yelp.com/biz/aperitivo-milwaukee?osq=Restaurants']

,Unnamed: 0,restaurant,link,Complete_links
90,90,Aperitivo,/biz/aperitivo-milwaukee?osq=Restaurants,https://www.yelp.com/biz/aperitivo-milwaukee?o...


In [65]:
restrest = ['Maki Yaki Japanese Grill', 'Proof Pizza', 'St Paul Fish Company', 'Aperitivo']

linklink = ['https://www.yelp.com/biz/maki-yaki-japanese-grill-milwaukee-3?osq=Restaurants', 'https://www.yelp.com/biz/proof-pizza-milwaukee-4?osq=Restaurants', 'https://www.yelp.com/biz/st-paul-fish-company-milwaukee?osq=Restaurants',
            'https://www.yelp.com/biz/aperitivo-milwaukee?osq=Restaurants']


In [39]:
import pyautogui

In [66]:
''' Now we scrape the reviews, this time we do not have to make use of the selenoum package, we onyl work with URLs which should make this process a bit easier'''

#we initialize a dataframe, but of course outside the loop
df1 = pd.DataFrame([],columns=['Restaurant', 'Review'])

enum_extract = enumerate(restrest)


failedurls = []

def getname():
    key = next(enum_extract)
    return str(key)


for l in linklink:
    url_yelp = l
    html = requests.get(url_yelp)
    soup2 = BeautifulSoup(html.content, 'lxml')
    while html.status_code == 200:
        try: 
            ## we search for the number of sites to loop through
            length=soup2.select('.css-e81eai')
            length = str(length)
            #we use a regular expression to filter out the total length of the review
            length = re.search('>(\d+) of (\d+)<', length)
            if length:
                length1 = length.group(1)
                length2 = int(length.group(2))
                
            if length2 > 20:
                length2 = 20

            reviews = []
            for i in range(0,length2*10,10):
                html = requests.get(url_yelp+'&start='+str(i))
                soup3 = BeautifulSoup(html.content, 'lxml')
                soupstring = str(soup3)
                #this is the pattern for all the reviews, it will find all matches so we don't have to worry about not grabbing one. Though we will have to clean this up later
                subsoup = re.findall('"markupDisplayName":(.+?),"voterText":', soupstring)
                reviews += subsoup
                time.sleep(5)

            #outside of the last loop, we create a dict, where we zip the lists, and then we append the dict to the list   
            restname = getname()
            keys = [restname]*int(len(reviews))
            review_list = list(zip(keys, reviews))

            df2=pd.DataFrame(review_list, columns=['Restaurant','Review'])

            frames= [df1, df2]
            df1 = pd.concat(frames)
            print(restname)
            time.sleep(3)  
                   
        except:
            length2 = 0
            restname = getname()
            print(f'No reviews for {restname}')
            time.sleep(5)
              
        #In case we have lots of reviews, we just scrape the first 200
         
    else:
        print(f'We got blocked for {url_yelp}')
        failedurls.append(url_yelp)
        
        pyautogui.moveTo(x=1145, y=14) 
        pyautogui.click()
        time.sleep(2)
        pyautogui.moveTo(x=1158, y=105) 
        pyautogui.moveTo(x=1350, y=105, duration = 3)
        time.sleep(2)
        pyautogui.moveTo(x=1350, y=220, duration=2)
        pyautogui.click()
        time.sleep(2)
        pyautogui.alert(text='Enter password', title='Enter password', button='OK')

        print('continue')
        #pyautogui.write('Ilemlbseicmzt@2501527uva', interval=0.25)
        time.sleep(1)
        #pyautogui.press('enter')
        time.sleep(6)

(0, 'Maki Yaki Japanese Grill')
(1, 'Proof Pizza')
(2, 'St Paul Fish Company')
(3, 'Aperitivo')


StopIteration: 

In [68]:
df1.tail()

,Restaurant,Review
62,"(3, 'Aperitivo')","""Laura S."",""displayLocation"":""Seattle, WA"",""al..."
63,"(3, 'Aperitivo')","""Francesca S."",""displayLocation"":""Morningside ..."
64,"(3, 'Aperitivo')","""Marielle S."",""displayLocation"":""Milwaukee, WI..."
65,"(3, 'Aperitivo')","""Sarah H."",""displayLocation"":""Milwaukee, WI"",""..."
66,"(3, 'Aperitivo')","""Justin S."",""displayLocation"":""Des Plaines, IL..."


In [69]:
df_7 = df1[df1['Restaurant'] != "(3, 'Aperitivo')"] #I accidentally added this Aperitivo restaurant twice

In [70]:
len(df_7)

181

In [71]:
df_7.to_csv('df_7.reviews.csv')

In [128]:
df_1= pd.read_csv('df_1.reviews.csv')
df_2= pd.read_csv('df_2.reviews.csv')
df_3= pd.read_csv('df_3.reviews.csv')
df_4= pd.read_csv('df_4.reviews.csv')
df_5 = pd.read_csv('df_5.reviews.csv')
df_6 = pd.read_csv('df_6.reviews.csv')
df_7 = pd.read_csv('df_7.reviews.csv')

In [129]:
df1 = pd.concat([df_1,df_2, df_3, df_4, df_5, df_6, df_7])
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [130]:
df1.head()

,Restaurant,Review
0,"(0, 'Swingin’ Door Exchange')","""KC and Shelly S."",""photoSrc"":""https://s3-medi..."
1,"(0, 'Swingin’ Door Exchange')","""Hart P."",""displayLocation"":""Gallatin, TN"",""al..."
2,"(0, 'Swingin’ Door Exchange')","""Warren D."",""displayLocation"":""Milwaukee, WI"",..."
3,"(0, 'Swingin’ Door Exchange')","""Daniela B."",""displayLocation"":""Brookfield, WI..."
4,"(0, 'Swingin’ Door Exchange')","""Ashley B."",""displayLocation"":""Chicago, IL"",""a..."


In [131]:
import regex as re

new_restaurants = df1.Restaurant.to_list()

'''stripping anything werid from the search term'''
def searchconcat(x):
    x = x.replace(',', '')\
    .replace("'","")\
    .replace('(','')\
    .replace(')','')\
    .replace('0', '')\
    .replace('1', '')\
    .replace('2', '')\
    .replace('3', '')\
    .replace('4', '')\
    .replace('5', '')\
    .replace('6', '')\
    .replace('7', '')\
    .replace('8', '')\
    .replace('9', '')\
    .strip()
    
    return str(x)



new_ii = []
for i in new_restaurants:
    new_i=searchconcat(i)
    new_ii.append(new_i)
    

df1['Restaurant'] = new_ii

In [132]:
df1.head()

,Restaurant,Review
0,Swingin’ Door Exchange,"""KC and Shelly S."",""photoSrc"":""https://s3-medi..."
1,Swingin’ Door Exchange,"""Hart P."",""displayLocation"":""Gallatin, TN"",""al..."
2,Swingin’ Door Exchange,"""Warren D."",""displayLocation"":""Milwaukee, WI"",..."
3,Swingin’ Door Exchange,"""Daniela B."",""displayLocation"":""Brookfield, WI..."
4,Swingin’ Door Exchange,"""Ashley B."",""displayLocation"":""Chicago, IL"",""a..."


In [133]:
''' Lets cut out the photos already, cause that would be too much to parse'''

df1[df1['Restaurant'] == 'Swingin’ Door Exchange']['Review'].values[1]

'"Hart P.","displayLocation":"Gallatin, TN","altText":"Hart P.","userUrl":"/user_details?userid=vLPnQDSSj18al51PJLnofA","partnerAlias":null,"friendCount":57,"photoCount":1339,"reviewCount":718,"eliteYear":2021},"comment":{"text":"Great experience. Indoor dining at the bar - patrons wear masks pretty well.<br&gt;<br&gt;With my schedule, I had to do the Friday night fish fry dinner otherwise I would not have another chance. Good, lots of side options. I chose the potato cakes and coleslaw as part of the dinner. The rye bread is very good. I picked up the carrots as an extra side and am so glad I did. The carrots are awesome. Sauteed brings out the carrot&amp;#39;s sweetness and the spices add excellent flavor. Something I will definitely have again as a side or shared plate/appetizer.<br&gt;<br&gt;Plenty of beer options and my local amber (Lakefront) paired well with the fish dinner.","language":"en"},"localizedDate":"8/11/2020","localizedDateVisited":null,"rating":5,"photos":[{"src":"ht

In [136]:
df1[['Review_1', 'Discard']] = df1['Review'].str.split('photos', 1, expand=True)

In [137]:
df1[['Discard', 'Review_2']] = df1['Discard'].str.split('displayName', 1, expand=True)

In [138]:
df1.pop('Review')
df1.pop('Discard')

0      ":[],"lightboxMediaItems":[],"photosUrl":"/biz...
1      ":[{"src":"https://s3-media0.fl.yelpcdn.com/bp...
2      ":[{"src":"https://s3-media0.fl.yelpcdn.com/bp...
3      ":[{"src":"https://s3-media0.fl.yelpcdn.com/bp...
4      ":[{"src":"https://s3-media0.fl.yelpcdn.com/bp...
                             ...                        
176    ":[],"lightboxMediaItems":[],"photosUrl":"/biz...
177    ":[],"lightboxMediaItems":[],"photosUrl":"/biz...
178    ":[],"lightboxMediaItems":[],"photosUrl":"/biz...
179    ":[],"lightboxMediaItems":[],"photosUrl":"/biz...
180    ":[],"lightboxMediaItems":[],"photosUrl":"/biz...
Name: Discard, Length: 12392, dtype: object

In [139]:
df1[df1['Review_2'] == None] 

,Restaurant,Review_1,Review_2


In [140]:
df1.tail()

,Restaurant,Review_1,Review_2
176,St Paul Fish Company,"""Laura S."",""displayLocation"":""Seattle, WA"",""al...",None
177,St Paul Fish Company,"""Francesca S."",""displayLocation"":""Morningside ...",None
178,St Paul Fish Company,"""Marielle S."",""displayLocation"":""Milwaukee, WI...",None
179,St Paul Fish Company,"""Sarah H."",""displayLocation"":""Milwaukee, WI"",""...",None
180,St Paul Fish Company,"""Justin S."",""displayLocation"":""Des Plaines, IL...",None


In [107]:
len(df1)

12392

In [111]:
df1.Restaurant.nunique()

223

In [112]:
df = pd.read_csv('mainlinks.csv')

In [113]:
df.restaurant.nunique()

238

In [146]:
''' Now we need to comapre the difference'''

old_rest = df.restaurant.to_list()

new_rest = df1.Restaurant.to_list()


#new_rest = df_mainlinks.restaurant.to_list()

#df_new = df1[~df1['Restaurant'].isin(old_rest)]

#df_new

No, List1 doesn't have all elements of the List2.


In [108]:
df1.to_csv('all_reviews.csv')